In [31]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict,Annotated,Literal
import operator
from dotenv import load_dotenv
from pydantic import BaseModel,Field
load_dotenv()

True

In [32]:
class value(TypedDict):
    current_value: int
    target_value: int
    iteratons: int
    status: Literal["yes","no"]
    result: str

In [ ]:
import random

def generate(state: value) -> value:
    current_value=random.randint(0,10)
    return {"current_value": current_value}


def increase(state:value) -> value:
    current_value=state["current_value"] +1
    iterations=state["iteratons"]+1
    return {"current_value": current_value, "iteratons": iterations}

def add_result(state:value) -> value:
    # result=
    return {"result": f"Reached target value {state['target_value']} in {state['iteratons']} iterations"}


def check_target(state:value) -> value:
    if state["current_value"] == state["target_value"]:
        print("returning yes")
        return {"status":"yes"}
    else:
        print("returning no")
        return {"status":"no"}
    
def router(state: value) -> Literal["yes", "no"]:
    if state["status"] == "yes":
        return "yes"
    else:
        return "no"

In [54]:
graph=StateGraph(value)

graph.add_node("generate",generate)
graph.add_node("check_target",check_target)
graph.add_node("increase",increase)
graph.add_node("add_result",add_result)


graph.add_edge(START,"generate")
graph.add_edge("generate","check_target")
graph.add_conditional_edges("check_target",router,{"yes":"add_result","no":"increase"})
graph.add_edge("increase","check_target")
graph.add_edge("add_result",END)

# graph.compile()

In [55]:
initial_state={"target_value":15,"iteratons":0,"status":"no","result":""}
workflow=graph.compile()
final_state=workflow.invoke(initial_state)

returning no
returning no
returning no
returning no
returning no
returning no
returning yes


In [56]:
final_state

{'current_value': 15,
 'target_value': 15,
 'iteratons': 6,
 'status': 'yes',
 'result': 'Reached target value 15 in 6 iterations'}